## setup

In [1]:
import requests
from pprint import pprint,pformat

elucidate_base_url = 'http://localhost:8080/annotation'
headers = {
    'Accept':'application/ld+json; profile="http://www.w3.org/ns/anno.jsonld"',
    'Content-Type': 'application/ld+json; profile="http://www.w3.org/ns/anno.jsonld"'
}

def get_etag(url:str)->str:
    return requests.get(url).headers['etag'][3:-1]

## create container

In [2]:
body = {
  "@context": [
    "http://www.w3.org/ns/anno.jsonld",
    "http://www.w3.org/ns/ldp.jsonld"
  ],
  "type": [
    "BasicContainer",
    "AnnotationCollection"
  ],
  "label": "A Container for Web Annotations"
}

response = requests.post(url=elucidate_base_url+"/w3c/",headers=headers,json=body)
container_uri = response.json()['id']

response.status_code

201

## read container

In [3]:
requests.get(container_uri).json()

{'@context': ['http://www.w3.org/ns/anno.jsonld',
  'http://www.w3.org/ns/ldp.jsonld'],
 'id': 'http://localhost:8080/annotation/w3c/93d51025-7e68-4e50-b55c-5426f3dc33e3/',
 'type': 'AnnotationCollection',
 'label': 'A Container for Web Annotations',
 'first': {'type': 'AnnotationPage',
  'as:items': {'@list': []},
  'partOf': 'http://localhost:8080/annotation/w3c/93d51025-7e68-4e50-b55c-5426f3dc33e3/',
  'startIndex': 0},
 'last': 'http://localhost:8080/annotation/w3c/93d51025-7e68-4e50-b55c-5426f3dc33e3/?page=0&desc=1',
 'total': 0}

## create annotation

In [4]:
annotation = {
  "@context": "http://www.w3.org/ns/anno.jsonld",
  "type": "Annotation",
  "body": {
    "type": "TextualBody",
    "value": "I like this page!"
  },
  "target": "http://www.example.com/index.html"
}

response = requests.post(
    url=container_uri,
    headers=headers,
    json=annotation)
annotation_uri = response.json()['id']

response.status_code

201

## read annotation

- W3C :

In [5]:
pprint(annotation_uri)
requests.get(annotation_uri).json()

'http://localhost:8080/annotation/w3c/93d51025-7e68-4e50-b55c-5426f3dc33e3/9e03358a-6c4b-440e-92e7-c40826e8e032'


{'@context': 'http://www.w3.org/ns/anno.jsonld',
 'id': 'http://localhost:8080/annotation/w3c/93d51025-7e68-4e50-b55c-5426f3dc33e3/9e03358a-6c4b-440e-92e7-c40826e8e032',
 'type': 'Annotation',
 'body': {'type': 'TextualBody', 'value': 'I like this page!'},
 'target': 'http://www.example.com/index.html'}

- OA :

In [6]:
oa_uri = annotation_uri.replace('/w3c/','/oa/')
pprint(oa_uri)
requests.get(oa_uri).json()

'http://localhost:8080/annotation/oa/93d51025-7e68-4e50-b55c-5426f3dc33e3/9e03358a-6c4b-440e-92e7-c40826e8e032'


{'@context': 'https://raw.githubusercontent.com/dlcs/elucidate-server/master/elucidate-server/src/main/resources/contexts/oa.jsonld',
 '@id': 'http://localhost:8080/annotation/oa/93d51025-7e68-4e50-b55c-5426f3dc33e3/9e03358a-6c4b-440e-92e7-c40826e8e032',
 '@type': 'oa:Annotation',
 'hasBody': {'@type': 'oa:TextualBody', 'value': 'I like this page!'},
 'hasTarget': 'http://www.example.com/index.html'}

## update annotation

In [7]:
etag = get_etag(annotation_uri)
new_annotation = {
  "@context": "http://www.w3.org/ns/anno.jsonld",
  "id": "http://example.org/w3c/mycontainer/anno1",
  "type": "Annotation",
  "created": "2015-01-31T12:03:45Z",
  "body": {
    "type": "TextualBody",
    "value": "I don't like this page!"
  },
  "target": "http://www.example.com/index.html"
}
update_headers = {'If-Match':etag}
update_headers.update(headers)
response = requests.put(url=annotation_uri,headers=update_headers,json=new_annotation)

response.status_code

200

In [8]:
requests.get(annotation_uri).json()

{'@context': 'http://www.w3.org/ns/anno.jsonld',
 'id': 'http://localhost:8080/annotation/w3c/93d51025-7e68-4e50-b55c-5426f3dc33e3/9e03358a-6c4b-440e-92e7-c40826e8e032',
 'type': 'Annotation',
 'created': '2015-01-31T12:03:45Z',
 'body': {'type': 'TextualBody', 'value': "I don't like this page!"},
 'target': 'http://www.example.com/index.html'}

In [9]:
oa_uri = annotation_uri.replace('w3c','oa')
pprint(oa_uri)
requests.get(oa_uri).json()

'http://localhost:8080/annotation/oa/93d51025-7e68-4e50-b55c-5426f3dc33e3/9e03358a-6c4b-440e-92e7-c40826e8e032'


{'@context': 'https://raw.githubusercontent.com/dlcs/elucidate-server/master/elucidate-server/src/main/resources/contexts/oa.jsonld',
 '@id': 'http://localhost:8080/annotation/oa/93d51025-7e68-4e50-b55c-5426f3dc33e3/9e03358a-6c4b-440e-92e7-c40826e8e032',
 '@type': 'oa:Annotation',
 'annotatedAt': '2015-01-31T12:03:45Z',
 'hasBody': {'@type': 'oa:TextualBody', 'value': "I don't like this page!"},
 'hasTarget': 'http://www.example.com/index.html'}

## annotation history

In [17]:
response = requests.get(annotation_uri)
for (k,v) in response.headers.items():
    print(f"{k} = {v}")

ETag = W/"b11bb656a4e1a7778950eea066dc8e2b"
Link = <http://www.w3.org/ns/ldp#Resource>; rel="type", <http://localhost:8080/annotation/w3c/services/history/93d51025-7e68-4e50-b55c-5426f3dc33e3/9e03358a-6c4b-440e-92e7-c40826e8e032/1>; rel="prev memento"; datetime="Thu, 18 Mar 2021 21:51:41 UTC"
Allow = PUT,GET,OPTIONS,HEAD,DELETE
Vary = Accept
Memento-Datetime = Thu, 18 Mar 2021 21:52:27 UTC
X-Content-Type-Options = nosniff
X-XSS-Protection = 1; mode=block
Cache-Control = no-cache, no-store, max-age=0, must-revalidate
Pragma = no-cache
Expires = 0
X-Frame-Options = DENY
Content-Type = application/ld+json;charset=UTF-8
Transfer-Encoding = chunked
Date = Thu, 18 Mar 2021 21:55:29 GMT


In [21]:
response.headers['link']

'<http://www.w3.org/ns/ldp#Resource>; rel="type", <http://localhost:8080/annotation/w3c/services/history/93d51025-7e68-4e50-b55c-5426f3dc33e3/9e03358a-6c4b-440e-92e7-c40826e8e032/1>; rel="prev memento"; datetime="Thu, 18 Mar 2021 21:51:41 UTC"'

## delete annotation

In [22]:
etag = get_etag(annotation_uri)
delete_headers = {'If-Match':etag}
delete_headers.update(headers)
response = requests.delete(url=annotation_uri,headers=delete_headers)

response.status_code

204

In [23]:
requests.get(annotation_uri).status_code

410